# load dask cluster with data
load a parquet dataset into a dask cluster

In [1]:
# nuclio: ignore
import nuclio

In [2]:
import os
import json
import numpy as np
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

from typing import List, Optional

def load_dask(
    context: MLClientCtx,
    src_data: str,
    dask_key: str = "dask_key",
    inc_cols: Optional[List[str]] = None,
    index_cols: Optional[List[str]] = None,
    dask_persist: bool = True,
    refresh_data: bool = True,
    scheduler_key: str = "scheduler"
) -> None:
    """Load dataset into an existing dask cluster
    
    dask jobs define the dask client parameters at the job level, this method will raise an error if no client is detected.
    
    :param context:         the function context
    :param src_data:        url of the data file or partitioned dataset as either
                            artifact DataItem, string, or path object (similar to 
                            pandas read_csv)
    :param dask_key:        destination key of data on dask cluster and artifact store
    :param inc_cols:        include only these columns (very fast)
    :param index_cols:      list of index column names (can be a long-running process)
    :param dask_persist:    (True) should the data be persisted (through the `client.persist` op)
    :param refresh_data:    (False) if the dask_key already exists in the dask cluster, this will 
                            raise an Exception.  Set to True to replace the existing cluster data.
    :param scheduler_key:   (scheduler) the dask scheduler configuration, json also logged as an artifact
    """
    if hasattr(context, "dask_client"):
        dask_client = context.dask_client
    else:
        raise Exception("a dask client was not found in the execution context")
    
    src_data = str(src_data)
    if isinstance(src_data, str):
        if os.path.isdir(src_data) or src_data.endswith("pq") or src_data.endswith("parquet"):
            df = dd.read_parquet(src_data)
        elif src_data.endswith("csv"):
            df = dd.read_csv(src_data)

    if dask_persist:
        df = dask_client.persist(df)
        if dask_client.datasets and dask_key in dask_client.datasets:
            dask_client.unpublish_dataset(dask_key)
        dask_client.publish_dataset(df, name=dask_key)
    
    if context:
        context.dask_client = dask_client
        
    # share the scheduler, whether data is persisted or not
    filepath = os.path.join(context.artifact_path, scheduler_key+".json")
    dask_client.write_scheduler_file(filepath)
    
    # we don't use log_dataset here until it can take into account
    # dask origin and apply dask describe.
    context.log_artifact(scheduler_key, local_path=scheduler_key+".json")

In [3]:
# nuclio: end-code

### mlconfig

In [4]:
from mlrun import mlconf

In [5]:
mlconf.dbpath = mlconf.dbpath or './'
mlconf.dbpath

'http://mlrun-api:8080'

In [6]:
vcs_branch = 'development'
base_vcs = f'https://raw.githubusercontent.com/mlrun/functions/{vcs_branch}/'

mlconf.hub_url = mlconf.hub_url or base_vcs + f'{name}/function.yaml'
mlconf.hub_url

'/User/repos/functions/{name}/function.yaml'

In [7]:
import os
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["V3IO_HOME"]}/artifacts'
mlconf.artifact_path

'/User/artifacts'

In [8]:
import os
TAG = os.environ['MLRUN_COMMIT']

### save

In [19]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dask", kind="dask", 
                      with_doc=True,
                      handler=load_dask,
                      image = f"mlrun/ml-models:{TAG}")

# add metadata (for templates and reuse)
fn.spec.description = "load dask cluster with data"
fn.metadata.categories = ["fileutils", "distributed"]
fn.metadata.labels = {"author": "yjb"}
fn.spec.remote = True
fn.spec.replicas = 4
fn.spec.max_replicas = 8
fn.spec.service_type = "NodePort"
fn.spec.image_pull_policy = 'Always'

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-28 19:58:39,758 saving function: load-dask, tag: latest
[mlrun] 2020-04-28 19:58:39,789 function spec saved to path: function.yaml


### test

In [20]:
from mlrun import import_function

func = import_function("hub://load_dask")

if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    func.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    func.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [21]:
from mlrun import NewTask, run_local

task_params = {
    "name":        "tasks load dask cluster with data",
    "params" : {
        "persist"      : True,
        "refresh_data" : True,
        "dask_key"     : "dask_key"}}

In [22]:
run = func.run(NewTask(**task_params), 
               handler=load_dask, 
               inputs={"src_data" : os.path.join(mlconf.artifact_path, 'iris.parquet') },
               artifact_path=mlconf.artifact_path)

[mlrun] 2020-04-28 19:58:39,849 starting run tasks load dask cluster with data uid=a6b4201ab2844cbb98874fb67c2f86f1  -> http://mlrun-api:8080
[mlrun] 2020-04-28 19:58:39,920 saving function: load-dask, tag: latest
[mlrun] 2020-04-28 19:58:44,622 trying dask client at: tcp://mlrun-load-dask-7ed5312a-6.default-tenant:8786
[mlrun] 2020-04-28 19:58:44,647 using remote dask scheduler (mlrun-load-dask-7ed5312a-6) at: tcp://mlrun-load-dask-7ed5312a-6.default-tenant:8786


[mlrun] 2020-04-28 19:58:44,708 log artifact scheduler at /User/artifacts/scheduler.json, size: None, db: Y

[mlrun] 2020-04-28 19:58:44,738 run ended with state 


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
,...7c2f86f1,0,Apr 28 19:58:39,completed,tasks load dask cluster with data,v3io_user=adminkind=daskowner=adminhost=jupyter-6b586bcb85-4f5j7,src_data,persist=Truerefresh_data=Truedask_key=dask_key,,scheduler


to track results use .show() or .logs() or in CLI: 
!mlrun get run a6b4201ab2844cbb98874fb67c2f86f1  , !mlrun logs a6b4201ab2844cbb98874fb67c2f86f1 
[mlrun] 2020-04-28 19:58:44,762 run executed, status=completed


In [23]:
func.status.to_dict()

{'scheduler_address': 'tcp://mlrun-load-dask-7ed5312a-6.default-tenant:8786',
 'cluster_name': 'mlrun-load-dask-7ed5312a-6',
 'node_ports': {'dashboard': 32185, 'scheduler': 31991}}

In [24]:
import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

### TODO: this client dash board wont work -- wrong port!

...even though its the correct client

In [26]:
client = Client(func.status.to_dict()['scheduler_address'])
client

Client Scheduler: tcp://mlrun-load-dask-7ed5312a-6.default-tenant:8786 Dashboard: http://mlrun-load-dask-7ed5312a-6.default-tenant:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.44 GB


In [27]:
list(client.list_datasets())

['dask_key']

In [28]:
client.datasets['dask_key']

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),labels
npartitions=1,,,,,
,float64,float64,float64,float64,float64
,...,...,...,...,...
